# Atmosphere tutorial

This notebook contains sample cases for the Eradiate preview session of Dec 7th 2021. In this tutorial, we introduce the capabilities of Eradiate's 1D model. We will explore the features and interfaces of different objects used to construct a scene, simulate radiative transfer and visualise results.

In [ ]:
# We load the Rich notebook extension for improved object inspection
%load_ext rich

# We set the Matplotlib inline backend to SVG for better figure quality
from matplotlib_inline import backend_inline
backend_inline.set_matplotlib_formats("svg")

# We import general processing and plotting libraries
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd

# We import the top-level Eradiate module
import eradiate

# For convenience, we alias a few components
from eradiate import unit_registry as ureg
import eradiate.scenes as ertsc
import eradiate.experiments as ertxp

# We'll use the correlated k-distribution mode in double precision
eradiate.set_mode("ckd_double")

# We activate a few convenient presets
eradiate.notebook.install()
eradiate.plot.set_style(rc={"figure.dpi": 96})
eradiate.config.progress = 1  # Restrict progress display to the highest level

## General introduction

Our scene consists of a one-dimensional atmosphere model and a uniform reflecting surface. Since we want results for a given instrument band, we selected the CKD mode. It implements the correlated *k*-distribution (CKD) absorption model, which consistutes a compromise between correctness and speed. CKD requirements are often not very well fulfilled, but it is still more accurate than a grey band model and (much) faster than monochromatic runs.

Let's start by configuring the surface. We'll use a Lambertian surface with default parameters.

In [ ]:
surface = ertsc.surface.LambertianSurface()
surface

Now, let's add a molecular atmosphere on top of it. Eradiate currently provides pre-computed CKD absorption data for the AFGL 1986 atmosphere model with the so-called *U.S. Standard* thermophysical property profile. We'll therefore create an atmosphere object leveraging this data.

In [ ]:
atmosphere = ertsc.atmosphere.MolecularAtmosphere.afgl_1986()
atmosphere

We also have to illuminate our scene. Eradiate simulates Sun illumination using a [perfectly directional model](https://eradiate.readthedocs.io/en/latest/rst/reference/generated/autosummary/eradiate.scenes.illumination.DirectionalIllumination.html) and ships with a number of [Solar irradiance spectra](https://eradiate.readthedocs.io/en/latest/rst/reference/generated/autosummary/eradiate.scenes.spectra.SolarIrradianceSpectrum.html). Although we will stick with the default spectrum (Thuillier, 2003), we could select another one. Available spectra are listed on the [*Solar irradiance spectrum* data guide](https://eradiate.readthedocs.io/en/latest/rst/user_guide/data/solar_irradiance_spectrum_data_sets.html).

We will set up our illumination to a local 15° zenith angle and 0° azimuth angle (note: [Eradiate uses a counter-clockwise azimuth orientation, with 0° aligned with the local *X*/East axis](https://eradiate.readthedocs.io/en/latest/rst/user_guide/conventions.html)).

In [ ]:
illumination = ertsc.illumination.DirectionalIllumination(
    zenith=15.0,
    azimuth=0.0,
)
illumination

The last bit we need is to define the measurement we want the Monte Carlo simulator to make. We will compute the top-of-atmosphere bidirectional reflectance factor (TOA BRF). This is done using a *distant radiometer*, an abstract measurement which, in practice, computes the radiance leaving the scene at an infinite distance. It is done with the [`MultiDistantMeasure` class](https://eradiate.readthedocs.io/en/latest/rst/reference/generated/autosummary/eradiate.scenes.measure.MultiDistantMeasure.html). This class can be constructed in several ways:

* one may pass directions manually using its basic constructor;
* one may specify directions using viewing angles using the `from_viewing_angles()` class method constructor.

We will use the second possibility, which feels more natural for Earth observation-related applications. The `from_viewing_angles()` has two main arguments `zeniths` and `azimuths`. By default, both must be arrays of the same size, each element being the zenith (resp. azimuth) value of a zenith-azimuth pair. If all pairs have the same zenith (resp. azimuth) value, then a scalar value may be passed as the zenith (resp. azimuth).

The second important parametrisation point of our measure is its spectral configuration. This includes:

* The instrument's spectral response function. Importantly, this will determine the spectral range the measure will process. Eradiate has a number of instrument spectral response functions to choose from, and we will select a rather [narrow band of Sentinel-2](https://sentinel.esa.int/web/sentinel/user-guides/sentinel-2-msi/document-library/-/asset_publisher/Wk0TKajiISaR/content/sentinel-2a-spectral-responses) (narrower bands are quicker to process).
* In CKD mode, the discretisation used for the spectral domain. Eradiate currently ships pre-computed gaseous absorption data for 1 nm and 10 nm-wide spectral bins. Narrower bins yield more accurate results, but they also require more computational resources to cover a given spectral range. For this example, we will use the 10 nm discretisation.

Finally, we have to define the number of radiance samples the Monte Carlo will accumulate to compute this BRF. The computational time is highly dependent on this number, and we will first set it to the default 1000—arguably a low number, which we can increase later on to reduce the amount of noise in our results.

In [ ]:
measure = ertsc.measure.MultiDistantMeasure.from_viewing_angles(
    id="toa_brf",
    zeniths=np.arange(-75, 76, 5),  # We cover the [-75°, 75°] range with 5° steps
    azimuths=0,  # The same value as the SAA so as to cover the principal plane
    spectral_cfg={
        "srf": "sentinel_2a-msi-5",
        "bin_set": "10nm",
    },
    spp=1000,
)
measure

Now, we can create an *experiment* instance. It will assemble our scene elements and provide simulation and post-processing interfaces.

In [ ]:
exp = ertxp.OneDimExperiment(
    surface=surface,
    atmosphere=atmosphere,
    illumination=illumination,
    measures=measure,
)
exp

Now, let's run our simulation. The `run()` method launches the computation, then chains it with the post-processing pipeline:

In [ ]:
exp.run()

We can now visualise our results. Let's first collect our result dataset for the `results` dictionary:

In [ ]:
results_intro = exp.results["toa_brf"]
results_intro

This dataset has many fields. The `_srf`-suffixed fields have the spectral response function applied, while their suffix-free counterparts have a spectral dependency.

In [ ]:
with plt.style.context({"lines.marker": ".", "lines.linestyle": ":"}):
    results_intro.brf_srf.squeeze(drop=True).plot(x="vza")
    plt.title(f"sza = {float(results_intro.sza)} {results_intro.sza.units}")
    plt.show()
    plt.close()

**Exercises for the interested reader**

1. Increase the surface reflectance to 0.85 and run the simulation again.
2. Increase the sample count ten-fold (to 10000) so as to reduce the Monte Carlo noise.
3. Change the surface reflectance and use the [Rahman-Pinty-Verstraete (RPV) model](https://eradiate.readthedocs.io/en/latest/rst/reference/generated/autosummary/eradiate.scenes.surface.RPVSurface.html). The BRF in the principal plane should feature a clear "hot spot", located to the right of the plot.
4. Change the Solar azimuth angle and reorient the measure accordingly so that it remains in the principal plane.
5. Modify the angular parametrisation of the measure to record radiance in a hemispherical plane orthogonal to the principal plane.

## Adjusting atmospheric parameters

### Adjusting chemical composition

Standard atmospheric profiles are defined with a given chemical composition and changing this in CKD modes is considerably complicated by the amount of work required to recompute CKD spectra. However, Eradiate includes an approximate *rescaling* method to adjust CKD spectra depending on the update chemical composition. This can be done using the `concentrations` of the [`MolecularAtmosphere.afgl_1986()`](https://eradiate.readthedocs.io/en/latest/rst/reference/generated/autosummary/eradiate.scenes.atmosphere.MolecularAtmosphere.html#eradiate.scenes.atmosphere.MolecularAtmosphere.afgl_1986) constructor.

The following snippet defines a simulation in Sentinel-2/MSI's band 8 with rescaled water vapour concentration (the surface is white and diffuse):

In [ ]:
water_concentration = 14.3 * ureg("kg/m^2")

exp = ertxp.OneDimExperiment(
    surface=ertsc.surface.LambertianSurface(reflectance=1.0),
    atmosphere=ertsc.atmosphere.MolecularAtmosphere.afgl_1986(
        concentrations={"H2O": water_concentration}
    ),
    illumination=ertsc.illumination.DirectionalIllumination(
        zenith=0.0, azimuth=0.0
    ),
    measures=ertsc.measure.MultiDistantMeasure.from_viewing_angles(
        id="toa_brf",
        zeniths=np.arange(-75, 76, 15),
        azimuths=0,
        spectral_cfg={
            "srf": "sentinel_2a-msi-8",
            "bin_set": "10nm",
        },
        spp=1000,
    ),
)
exp.run()

results_chemscale = exp.results["toa_brf"]

with plt.style.context({"lines.marker": ".", "lines.linestyle": ":"}):
    results_chemscale.brf_srf.squeeze(drop=True).plot(x="vza")
plt.show()
plt.close()

**Exercises for the interested reader**

1. Repeat this simulation with the following concentration values: `[4.20, 14.3, 41.6] * ureg("kg/m^2")`. Plot the results and check that water vapour significantly decreases the transmittance within that band.
2. Switch to band 8A and see that water vapour concentration has much less impact there.
3. Make a simulation for a given Sentinel-2 band 8A, then set the BRF to a uniform value, still covering the extent of band 8A.

In [ ]:
# Solution to exercises

results_chemscale_exercises = []
water_concentrations = [4.20, 14.3, 41.6] * ureg("kg/m^2")  # These are the min, default and max values

for water_concentration in water_concentrations:
    exp = ertxp.OneDimExperiment(
        surface=ertsc.surface.LambertianSurface(reflectance=1.0),
        atmosphere=ertsc.atmosphere.MolecularAtmosphere.afgl_1986(
            concentrations={"H2O": water_concentration}
        ),
        illumination=ertsc.illumination.DirectionalIllumination(
            zenith=0.0, azimuth=0.0
        ),
        measures=ertsc.measure.MultiDistantMeasure.from_viewing_angles(
            id="toa_brf",
            zeniths=np.arange(-75, 76, 15),
            azimuths=0,
            spectral_cfg={
                "srf": "sentinel_2a-msi-8a",
                #"srf": {"type": "interpolated", "wavelengths": [847, 881], "values": [1, 1]},
                "bin_set": "10nm",
            },
            spp=1000,
        ),
    )
    exp.run()
    results_chemscale_exercises.append(exp.results["toa_brf"])

# Aggregate results in a dataset for easier plotting
ds_chemscale_exercises = xr.concat(
    results_chemscale_exercises, dim="water_concentration"
).reindex(
    {
        "water_concentration": pd.Index(water_concentrations.magnitude, name="water_concentration"),

    }
)
ds_chemscale_exercises["water_concentration"].attrs = {
    "long_name": "H$_2$O",
    "units": "kg/m²"
}
# ds

# Plot the results
with plt.style.context({"lines.marker": ".", "lines.linestyle": ":"}):
    ds_chemscale_exercises.brf_srf.squeeze(drop=True).plot(x="vza", hue="water_concentration")
plt.show()
plt.close()

### Adding aerosols

Eradiate's 1D simulator can account for the presence of aerosols in the atmosphere. The basic component for this is the [`ParticleLayer` class](https://eradiate.readthedocs.io/en/latest/rst/reference/generated/autosummary/eradiate.scenes.atmosphere.ParticleLayer.html). It is parametrised by:

* an extent, defined by its `bottom` and `top` parameters;
* a number of layers (`n_layers`) the particle layer is discretised;
* a particle spatial density distribution (*e.g.* uniform or gaussian)
* an optical thickness value at 550 nm.

An optical property dataset controlling the variation of the extinction coefficient, single scattering albedo and phase function can also be specified, but Eradiate currently only ships default testing data. Additional datasets will be added soon.

In [ ]:
particle_layer = ertsc.atmosphere.ParticleLayer(
    bottom=1 * ureg.km,
    top=2 * ureg.km,
    n_layers=16,
    tau_550=0.5,
)
particle_layer

We can assemble this together with the molecular component of our atmosphere using the [`HeterogeneousAtmosphere` container](https://eradiate.readthedocs.io/en/latest/rst/reference/generated/autosummary/eradiate.scenes.atmosphere.HeterogeneousAtmosphere.html):

In [ ]:
atmosphere = ertsc.atmosphere.HeterogeneousAtmosphere(
    molecular_atmosphere=ertsc.atmosphere.MolecularAtmosphere.afgl_1986(),
    particle_layers=[particle_layer]
)
atmosphere

**Exercises for the interested reader**

1. Simulate the TOA BRF for Sentinel-2 (choose a band) for a scene composed of:
   * A default AFGL 1986 atmosphere (U.S. Standard profile)
   * An aerosol layer of your choice
   * A surface of your choice
2. Compare the results with a simulation for a scene without the aerosol layer.